Variational AutoEncoder (VAE) with MNIST
========================================
---
Formation Introduction au Deep Learning  (FIDLE) - S. Arias, E. Maldonado, JL. Parouty - CNRS/SARI/DEVLOG - 2020  

## Episode 1 - Train a model
 - Defining a VAE model
 - Build the model
 - Train it
 - Follow the learning process with Tensorboard


## Step 1 - Init python stuff

In [1]:
import numpy as np
import sys, importlib

import modules.vae
import modules.loader_MNIST

from modules.vae          import VariationalAutoencoder
from modules.loader_MNIST import Loader_MNIST

VariationalAutoencoder.about()


FIDLE 2020 - Variational AutoEncoder (VAE)
TensorFlow version   : 2.0.0
VAE version          : 1.24


## Step 2 - Get data

In [2]:
(x_train, y_train), (x_test, y_test) = Loader_MNIST.load()

Dataset loaded.
Normalized.
Reshaped to (60000, 28, 28, 1)


## Step 3 - Get VAE model

In [3]:
tag = '001'

input_shape = (28,28,1)
z_dim       = 2
verbose     = 0

encoder= [ {'type':'Conv2D',          'filters':32, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'}
         ]

decoder= [ {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':1,  'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'sigmoid'}
         ]

vae = modules.vae.VariationalAutoencoder(input_shape    = input_shape, 
                                         encoder_layers = encoder, 
                                         decoder_layers = decoder,
                                         z_dim          = z_dim, 
                                         verbose        = verbose,
                                         run_tag        = tag)
vae.save(model=None)

Model initialized.
Outputs will be in  : ./run/001
Config saved in     : ./run/001/models/vae_config.json


## Step 4 - Compile it

In [4]:
learning_rate = 0.0005
r_loss_factor = 1000

vae.compile(learning_rate, r_loss_factor)

Compiled.
Optimizer is Adam with learning_rate=0.0005


## Step 5 - Train

In [5]:
batch_size        = 100
epochs            = 100
image_periodicity = 1      # for each epoch
chkpt_periodicity = 2      # for each epoch
initial_epoch     = 0
k_size            = 1      # 1 mean using 100% of the dataset

In [6]:
vae.train(x_train,
          x_test,
          batch_size        = batch_size, 
          epochs            = epochs,
          image_periodicity = image_periodicity,
          chkpt_periodicity = chkpt_periodicity,
          initial_epoch     = initial_epoch,
          k_size            = k_size
         )

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 10s 172us/sample - loss: 69.2860 - vae_r_loss: 66.7599 - vae_kl_loss: 2.5261 - val_loss: 53.5040 - val_vae_r_loss: 49.8820 - val_vae_kl_loss: 3.6220
Epoch 2/100
60000/60000 [==============================] - 7s 124us/sample - loss: 51.8624 - vae_r_loss: 47.9126 - vae_kl_loss: 3.9498 - val_loss: 50.5979 - val_vae_r_loss: 46.4222 - val_vae_kl_loss: 4.1757
Epoch 3/100
60000/60000 [==============================] - 7s 123us/sample - loss: 49.8990 - vae_r_loss: 45.5683 - vae_kl_loss: 4.3307 - val_loss: 49.3813 - val_vae_r_loss: 44.9857 - val_vae_kl_loss: 4.3957
Epoch 4/100
60000/60000 [==============================] - 8s 132us/sample - loss: 48.8134 - vae_r_loss: 44.2657 - vae_kl_loss: 4.5477 - val_loss: 48.3869 - val_vae_r_loss: 43.7678 - val_vae_kl_loss: 4.6190
Epoch 5/100
60000/60000 [==============================] - 8s 125us/sample - loss: 48.0753 - vae_r_loss: 43.4002 - vae_kl